<a href="https://colab.research.google.com/github/kazuki-de/git_reserch_code/blob/master/Autoencoder_tensorflow2.0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

!pip install -q tensorflow-gpu==2.0.0-beta1
import tensorflow as tf

from tensorflow.keras.layers import Dense, Flatten, Conv2D, Activation , Softmax, MaxPool2D,UpSampling2D
from tensorflow.keras import Model
import datetime
#import tensorflow as tf
import numpy as np
import pandas as pd
from PIL import Image

In [8]:
#x = tf.Constant(1,shape=(),dtype=tf.int32)
tf.executing_eagerly()

True

In [7]:
from google.colab import drive
drive.mount("/content/drive")


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [10]:
cd  /content/drive/My Drive/logs

/content/drive/My Drive/logs


In [0]:
dataset = tf.data.TextLineDataset("/content/drive/My Drive/2048_tissue_choice/sumpling_20csv")

In [12]:
dataset

<TextLineDatasetV2 shapes: (), types: tf.string>

In [0]:
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = 'logs/gradient_tape/' + current_time + '/train'
test_log_dir = 'logs/gradient_tape/' + current_time + '/test'
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
test_summary_writer = tf.summary.create_file_writer(test_log_dir)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="/content/drive/My Drive/logs/gradient_tape", histogram_freq=1)

In [0]:
path = "/content/drive/My Drive/2048pix_tissue_choice/sumpling_20.csv"

In [0]:
class MyModel1(Model):
    def __init__(self):
        super(MyModel1, self).__init__()
        self.Conv1 = Conv2D(4,(7,7),padding="same", activation='relu')
        self.Maxpool1 = MaxPool2D((2, 2), padding='same')
        self.Conv2 = Conv2D(8,(5,5), padding="same",activation='relu')
        self.Maxpool2 = MaxPool2D((2, 2), padding='same')
        self.Conv3 = Conv2D(16,(3,3),padding="same", activation='relu')
        self.Maxpool3 = MaxPool2D((2, 2), padding='same')
        
        self.Conv4 = Conv2D(16,(3,3),padding="same", activation='relu')
        self.Upsumpl1 = UpSampling2D((2, 2))
        self.Conv5 = Conv2D(8,(5,5), padding='same',activation='relu')
        self.Upsumpl2 = UpSampling2D((2, 2))
        self.Conv6 = Conv2D(4,(7,7), padding='same',activation='relu')
        self.Upsumpl3 = UpSampling2D((2, 2))
        self.Conv7 = Conv2D(3,(3,3), padding='same',activation='relu')
    
    def call(self, x):
        x = self.Conv1(x)
        x = self.Maxpool1(x)
        x = self.Conv2(x)
        x = self.Maxpool2(x)
        x = self.Conv3(x)
        x = self.Maxpool3(x)
        x = self.Conv4(x)
        x = self.Upsumpl1(x)
        x = self.Conv5(x)
        x = self.Upsumpl2(x)
        x = self.Conv6(x)
        x = self.Upsumpl3(x)
        return self.Conv7(x)

model1 = MyModel1()

In [0]:
#損失関数
loss_object = tf.keras.losses.MeanSquaredError()
#最適化手法
optimizer = tf.keras.optimizers.Adam()

#評価関数
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.CategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.CategoricalAccuracy(name='test_accuracy')

In [0]:
@tf.function
def train_step(image):
    with tf.GradientTape() as tape:
        predictions = model1(image)
        #softmax = Activation('softmax')(predictions)
        loss = loss_object(image, predictions)
        #VAT_loss = VAT_KL(image,model2)
        #print(loss.numpy())
        #print(loss.shape)
        #loss_add = loss + VAT_loss
    gradients = tape.gradient(loss, model1.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model1.trainable_variables))

    train_loss(loss)
    #train_accuracy(label, )


@tf.function
def test_step(image):
    predictions = model1(image)
    #softmax = Activation('softmax')(predictions)
    t_loss = loss_object(image, predictions)

    test_loss(t_loss)
    #test_accuracy(label, softmax)


In [0]:
path = "/content/drive/My Drive/2048pix_tissue_choice/sumpling_20.csv"
def generater_train():
    file_path1 = pd.read_csv("/content/drive/My Drive/2048pix_tissue_choice/tran_6000.csv")
    
    # LabelEncode(classをint型に変換)するためのdict\n",
    classes =["PI","PP","TRU"]
    classes = {v: i for i, v in enumerate(sorted(classes))}
    #file_path1 = pd.read_csv(file_path)
    length = len(file_path1)
    for i in range(length):
        path = file_path1.loc[i]["path1"]
        with Image.open(path) as f:
                tmp_image = np.asarray(f, dtype=np.float32)/255
        label = file_path1.loc[i]["type"]
        tmp_label = tf.keras.utils.to_categorical(classes[label],len(classes))
        #print(tmp_labels, np.mean(tmp_images, axis=(1,2,3)))
        yield tmp_image, tmp_label


In [0]:
def generater_val():
    file_path1 = pd.read_csv("/content/drive/My Drive/2048pix_tissue_choice/val_1000.csv")
    
    # LabelEncode(classをint型に変換)するためのdict\n",
    classes =["PI","PP","TRU"]
    classes = {v: i for i, v in enumerate(sorted(classes))}
    #file_path1 = pd.read_csv(file_path)
    length = len(file_path1)
    for i in range(length):
        path = file_path1.loc[i]["path1"]
        with Image.open(path) as f:
                tmp_image = np.asarray(f, dtype=np.float32)/255
        label = file_path1.loc[i]["type"]
        tmp_label = tf.keras.utils.to_categorical(classes[label],len(classes))
        #print(tmp_labels, np.mean(tmp_images, axis=(1,2,3)))
        yield tmp_image, tmp_label


In [0]:
EPOCHS = 200
#model = MyModel()
batchsize = 10
dataset_t = tf.data.Dataset.from_generator(generater_train, (tf.float32, tf.float32)).batch(20)
dataset_v = tf.data.Dataset.from_generator(generater_val, (tf.float32, tf.float32)).batch(20)

for epoch in range(EPOCHS):
    for batch, (x_t,x_l) in enumerate(dataset_t):
        train_step(x_t)
        print(batch)
    #with train_summary_writer.as_default():
        #tf.summary.scalar('loss', train_loss.result(), step=epoch)
        #tf.summary.scalar('accuracy', train_accuracy.result(), step=epoch)
    
    
    for batch, (y_t,y_l) in enumerate(dataset_t):
        test_step(y_t)
    #with test_summary_writer.as_default():
        #tf.summary.scalar('loss', test_loss.result(), step=epoch)
        #tf.summary.scalar('accuracy', test_accuracy.result(), step=epoch)

    template = 'Epoch {}, Loss: {}, Test Loss: {}'
    print (template.format(epoch+1,
                         train_loss.result(),
                         #train_accuracy.result()*100,
                         test_loss.result()
                         #test_accuracy.result()*100)
                          ))
    
#train_loss.reset_states()
#test_loss.reset_states()
#train_accuracy.reset_states()
#test_accuracy.reset_states()

In [0]:
def our_generator():
    for i in range(1000):
      x = np.random.rand(28,28)
      y = np.random.randint(1,10, size=1)
      yield x,y
    
dataset = tf.data.Dataset.from_generator(our_generator, (tf.float32, tf.int16))

In [102]:
dataset_t = tf.data.Dataset.from_generator(generater_train, (tf.float32, tf.float32))
for batch, (x,y) in enumerate(dataset_t.batch(10)):
    print(batch)
    print(y)

UnknownError: ignored